In [ ]:
import re, os, psycopg2, datetime, importlib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
#from dotenv import load_dotenv
import segmentação.inicializador as init

importlib.reload(init)

service, options = init.start_driver()

year = 2006

# Inicia-se a instância do Chrome WebDriver com as definidas 'options' e 'service', basicamente o driver É o google chrome.
for i in range(20):
    year = year + 1
    print(f'Coletando dados do ano {year}...')
    driver = init.go_to_site(service, options, f'https://www.boxofficemojo.com/year/world/{year}/')

    table = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
        (By.XPATH, "//table[@class='a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated scrolling-data-table']/tbody/tr/td/a[@href]")))
    links = [element.get_attribute("href") for element in table]

    lista_geral = []

    # Clica no primeiro link da lista e abre a página correspondente
    for link in links:

        driver.get(link)

        # pega o link do sumário (que abre a outra página com as informações que queremos)
        try:
            title_summary_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, "//div[@id='title-summary-refiner']/a")))
        except:
            continue
        # espera até que o link esteja presente e clica nele
        try:
            title_summary_link.click()
        except:
            print(f'Erro ao clicar no link: {link}')
            continue

        info_dict = {}
        # coleta do título, resumo e bilheteria (nacional, internacional e mundial)
        info_dict['movie_title'] = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, "//h1[@class='a-size-extra-large']"))).text
        info_dict['movie_summary'] = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, "//span[@class='a-size-medium']"))).text
        gross = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
            (By.XPATH, "//div[@class='a-section a-spacing-none mojo-performance-summary-table']/div/span[@class='a-size-medium a-text-bold']")))
        if gross[0].text == '–':
            info_dict['international_gross'] = int(gross[1].text.replace('$', '').replace(',', ''))
            info_dict['total_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))
        elif gross[1].text == '–':     
            info_dict['domestic_gross'] = int(gross[0].text.replace('$', '').replace(',', ''))
            info_dict['total_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))
        else:
            info_dict['domestic_gross'] = int(gross[0].text.replace('$', '').replace(',', ''))
            info_dict['international_gross'] = int(gross[1].text.replace('$', '').replace(',', ''))
            info_dict['total_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))

        # coleta do ano de lançamento, classificação etária, gênero, data de lançamento e duração
        table_info = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
        (By.XPATH, "//div[@class='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile']/div"))) 

        # essa outra tabela vem separada por \n, então precisamos separar as linhas
        # e depois separar por chave e valor (achei mais simples assim)
        for i in table_info:
            lines = i.text.split('\n')
            if len(lines) >= 2:
                key = lines[0].strip()
                value = lines[1].strip()
                info_dict[key] = value

        # ajuste de data para datetime (imagino que vá facilitar no banco de dados)
        # Remove the parenthesis and extra info
        date_str = info_dict['Earliest Release Date'].split('(')[0].strip()

        # Map Portuguese month names to English
        month_map = {
            'Janeiro': 'January', 'Fevereiro': 'February', 'Março': 'March', 'Abril': 'April',
            'Maio': 'May', 'Junho': 'June', 'Julho': 'July', 'Agosto': 'August',
            'Setembro': 'September', 'Outubro': 'October', 'Novembro': 'November', 'Dezembro': 'December'
        }

        for pt, en in month_map.items():
            if pt in date_str:
                date_str = date_str.replace(pt, en)
                break

        info_dict['Earliest Release Date'] = datetime.datetime.strptime(date_str, '%B %d, %Y').date().isoformat()

        # domestic open (ajuste para int e remover o $ e ,)
        try:
            info_dict['Domestic Opening'] = int(info_dict['Domestic Opening'].replace('$', '').replace(',', ''))
        except KeyError:
            pass

        if 'Budget' in info_dict:
            info_dict['Budget'] = int(info_dict['Budget'].replace('$', '').replace(',', ''))



        # running time (ajuste para int e remover o hr e min e transformar em minutos)
        try:
            split_running_time = info_dict['Running Time'].split(' ')
        except KeyError:
            info_dict['Running Time'] = None

        try:
            liquid_running_time = {split_running_time[1]: split_running_time[0], split_running_time[3]: split_running_time[2]}
            minutes = True
        except:
            try:
                liquid_running_time = {split_running_time[1]: split_running_time[0]}
                minutes = False
            except:
                minutes = False

        if minutes:
            info_dict['Running Time'] = int(liquid_running_time['hr']) * 60 + int(liquid_running_time['min'])
        else:
            try:
                info_dict['Running Time'] = int(liquid_running_time['hr']) * 60
            except:
                try:
                    info_dict['Running Time'] = int(liquid_running_time['min'])
                except KeyError:
                    info_dict['Running Time'] = None

        info_dict.pop('IMDbPro', None)

        lista_geral.append(info_dict)

    df = pd.DataFrame(lista_geral)
    df.pop('Budget')
    #df.pop('MPAA')

    df.rename(columns={'Domestic Distributor': 'domestic_distributor'}, inplace=True)
    df.rename(columns={'Domestic Opening': 'domestic_opening'}, inplace=True)
    df.rename(columns={'Earliest Release Date': 'release_date'}, inplace=True)
    df.rename(columns={'Running Time': 'running_time'}, inplace=True)
    df.rename(columns={'Genres': 'genres'}, inplace=True)
    df.rename(columns={'MPAA': 'indicative_rating'}, inplace=True)

    df = df[['movie_title', 'movie_summary', 'release_date', 'genres', 'indicative_rating', 'running_time', 'domestic_gross', 'international_gross', 'total_gross', 'domestic_opening', 'domestic_distributor']]

    df.to_csv(f'worldwide_box_office_{year}.csv', index=False)

    driver.quit()

Coletando dados do ano 2007...
Coletando dados do ano 2008...


KeyboardInterrupt: 

In [217]:
link

'https://www.boxofficemojo.com/releasegroup/gr2401456645/?ref_=bo_ydw_table_193'

In [ ]:
import re, os, psycopg2, datetime, importlib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#from dotenv import load_dotenv
import segmentação.inicializador as init

importlib.reload(init)

service, options = init.start_driver()

# Inicia-se a instância do Chrome WebDriver com as definidas 'options' e 'service', basicamente o driver É o google chrome.
year = '1989'
driver = init.go_to_site(service, options, f'https://www.boxofficemojo.com/year/world/{year}/')

table = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
    (By.XPATH, "//table[@class='a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated scrolling-data-table']/tbody/tr/td/a[@href]")))
links = [element.get_attribute("href") for element in table]

lista_geral = []

In [145]:
table = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
    (By.XPATH, "//table[@class='a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated scrolling-data-table']/tbody/tr/td/a[@href]")))
links = [element.get_attribute("href") for element in table]

lista_geral = []

In [193]:
# Clica no primeiro link da lista e abre a página correspondente
for link in links:

    driver.get(link)

    # pega o link do sumário (que abre a outra página com as informações que queremos)
    try:
        title_summary_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, "//div[@id='title-summary-refiner']/a")))
    except:
       continue
    # espera até que o link esteja presente e clica nele
    title_summary_link.click()

    info_dict = {}
    # coleta do título, resumo e bilheteria (nacional, internacional e mundial)
    info_dict['movie_title'] = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, "//h1[@class='a-size-extra-large']"))).text
    info_dict['movie_summary'] = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, "//span[@class='a-size-medium']"))).text
    gross = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
        (By.XPATH, "//div[@class='a-section a-spacing-none mojo-performance-summary-table']/div/span[@class='a-size-medium a-text-bold']")))
    if gross[0].text == '–':
        info_dict['international_gross'] = int(gross[1].text.replace('$', '').replace(',', ''))
        info_dict['total_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))
    elif gross[1].text == '–':     
        info_dict['domestic_gross'] = int(gross[0].text.replace('$', '').replace(',', ''))
        info_dict['total_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))
    else:
        info_dict['domestic_gross'] = int(gross[0].text.replace('$', '').replace(',', ''))
        info_dict['international_gross'] = int(gross[1].text.replace('$', '').replace(',', ''))
        info_dict['total_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))

    # coleta do ano de lançamento, classificação etária, gênero, data de lançamento e duração
    table_info = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
    (By.XPATH, "//div[@class='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile']/div"))) 

    # essa outra tabela vem separada por \n, então precisamos separar as linhas
    # e depois separar por chave e valor (achei mais simples assim)
    for i in table_info:
        lines = i.text.split('\n')
        if len(lines) >= 2:
            key = lines[0].strip()
            value = lines[1].strip()
            info_dict[key] = value

    # ajuste de data para datetime (imagino que vá facilitar no banco de dados)
    # Remove the parenthesis and extra info
    date_str = info_dict['Earliest Release Date'].split('(')[0].strip()

    # Map Portuguese month names to English
    month_map = {
        'Janeiro': 'January', 'Fevereiro': 'February', 'Março': 'March', 'Abril': 'April',
        'Maio': 'May', 'Junho': 'June', 'Julho': 'July', 'Agosto': 'August',
        'Setembro': 'September', 'Outubro': 'October', 'Novembro': 'November', 'Dezembro': 'December'
    }

    for pt, en in month_map.items():
        if pt in date_str:
            date_str = date_str.replace(pt, en)
            break

    info_dict['Earliest Release Date'] = datetime.datetime.strptime(date_str, '%B %d, %Y').date().isoformat()

    # domestic open (ajuste para int e remover o $ e ,)
    try:
        info_dict['Domestic Opening'] = int(info_dict['Domestic Opening'].replace('$', '').replace(',', ''))
    except KeyError:
        pass

    if 'Budget' in info_dict:
        info_dict['Budget'] = int(info_dict['Budget'].replace('$', '').replace(',', ''))



    # running time (ajuste para int e remover o hr e min e transformar em minutos)
    split_running_time = info_dict['Running Time'].split(' ')

    try:
        liquid_running_time = {split_running_time[1]: split_running_time[0], split_running_time[3]: split_running_time[2]}
        minutes = True
    except:
        liquid_running_time = {split_running_time[1]: split_running_time[0]}
        minutes = False

    if minutes:
        info_dict['Running Time'] = int(liquid_running_time['hr']) * 60 + int(liquid_running_time['min'])
    else:
        info_dict['Running Time'] = int(liquid_running_time['hr']) * 60

    info_dict.pop('IMDbPro', None)

    lista_geral.append(info_dict)

    


In [107]:
gross = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
        (By.XPATH, "//div[@class='a-section a-spacing-none mojo-performance-summary-table']/div/span[@class='a-size-medium a-text-bold']")))

In [186]:
lista_geral

[{'movie_title': 'Batman (1989)',
  'movie_summary': 'The Dark Knight of Gotham City begins his war on crime with his first major enemy being Jack Napier, a criminal who becomes the clownishly homicidal Joker.',
  'domestic_gross': 251409241,
  'international_gross': 160160000,
  'total_gross': 411569241,
  'Domestic Distributor': 'Warner Bros.',
  'Domestic Opening': 40489746,
  'Budget': 35000000,
  'Earliest Release Date': '1989-06-23',
  'Running Time': 126,
  'Genres': 'Action Adventure'},
 {'movie_title': 'Indiana Jones and the Last Crusade (1989)',
  'movie_summary': 'In 1938, after his father goes missing while pursuing the Holy Grail, Indiana Jones finds himself up against the Nazis again to stop them from obtaining its powers.',
  'domestic_gross': 197171806,
  'international_gross': 277000000,
  'total_gross': 474171806,
  'Domestic Distributor': 'Paramount Pictures',
  'Domestic Opening': 29355021,
  'Budget': 48000000,
  'Earliest Release Date': '1989-05-24',
  'MPAA': 'PG

In [8]:
title_summary_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "//div[@id='title-summary-refiner']/a")))
title_summary_link.click()

In [31]:
info_dict = {}
info_dict['movie_title'] = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "//h1[@class='a-size-extra-large']"))).text
info_dict['movie_summary'] = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "//span[@class='a-size-medium']"))).text
gross = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
    (By.XPATH, "//div[@class='a-section a-spacing-none mojo-performance-summary-table']/div/span[@class='a-size-medium a-text-bold']/span[@class='money']")))
info_dict['domestic_gross'] = int(gross[0].text.replace('$', '').replace(',', ''))
info_dict['international_gross'] = int(gross[1].text.replace('$', '').replace(',', ''))
info_dict['worldwide_gross'] = int(gross[2].text.replace('$', '').replace(',', ''))

In [33]:
info_dict

{'movie_title': 'Top Gun (1986)',
 'movie_summary': 'The Top Gun Naval Fighter Weapons School is where the best of the best train to refine their elite flying skills. When hotshot fighter pilot Maverick is sent to the school, his reckless attitude and cocky demeanor put him at odds with the other pilots, especially the cool and collected Iceman.',
 'domestic_gross': 180258178,
 'international_gross': 177030000,
 'worldwide_gross': 357288178,
 'Domestic Distributor': 'Paramount Pictures',
 'Domestic Opening': '$8,193,052',
 'Earliest Release Date': 'Maio 16, 1986 (Domestic)',
 'Running Time': '1 hr 49 min',
 'Genres': 'Action Drama',
 'IMDbPro': 'See more details at IMDbPro'}

In [11]:
table_info = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
    (By.XPATH, "//div[@class='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile']/div"))) 

In [47]:
for i in table_info:
    lines = i.text.split('\n')
    if len(lines) >= 2:
        key = lines[0].strip()
        value = lines[1].strip()
        info_dict[key] = value

print(info_dict)

{'movie_title': 'Top Gun (1986)', 'movie_summary': 'The Top Gun Naval Fighter Weapons School is where the best of the best train to refine their elite flying skills. When hotshot fighter pilot Maverick is sent to the school, his reckless attitude and cocky demeanor put him at odds with the other pilots, especially the cool and collected Iceman.', 'domestic_gross': 180258178, 'international_gross': 177030000, 'worldwide_gross': 357288178, 'Domestic Distributor': 'Paramount Pictures', 'Domestic Opening': '$8,193,052', 'Earliest Release Date': 'Maio 16, 1986 (Domestic)', 'Running Time': '1 hr 49 min', 'Genres': 'Action Drama', 'IMDbPro': 'See more details at IMDbPro'}


In [48]:
# Remove the parenthesis and extra info
date_str = info_dict['Earliest Release Date'].split('(')[0].strip()

# Map Portuguese month names to English
month_map = {
    'Janeiro': 'January', 'Fevereiro': 'February', 'Março': 'March', 'Abril': 'April',
    'Maio': 'May', 'Junho': 'June', 'Julho': 'July', 'Agosto': 'August',
    'Setembro': 'September', 'Outubro': 'October', 'Novembro': 'November', 'Dezembro': 'December'
}

for pt, en in month_map.items():
    if pt in date_str:
        date_str = date_str.replace(pt, en)
        break

info_dict['Earliest Release Date'] = datetime.datetime.strptime(date_str, '%B %d, %Y').date().isoformat()


In [53]:
info_dict['Domestic Opening'] = int(info_dict['Domestic Opening'].replace('$', '').replace(',', ''))

In [ ]:
split_running_time = info_dict['Running Time'].split(' ')

In [ ]:
split_running_time = info_dict['Running Time'].split(' ')

try:
    minutes = True
    liquid_running_time = {split_running_time[1]: split_running_time[0], split_running_time[3]: split_running_time[2]}
except:
    no_minutes = False
    liquid_running_time = {split_running_time[1]: split_running_time[0]}

if minutes:
    info_dict['Running Time'] = int(liquid_running_time['hr']) * 60 + int(liquid_running_time['min'])
else:
    info_dict['Running Time'] = int(liquid_running_time['hr']) * 60

info_dict.pop('IMDbPro', None)


In [76]:
if minutes:
    info_dict['Running Time'] = int(liquid_running_time['hr']) * 60 + int(liquid_running_time['min'])
else:
    info_dict['Running Time'] = int(liquid_running_time['hr']) * 60

In [81]:
info_dict.pop('IMDbPro', None)

'See more details at IMDbPro'

In [82]:
info_dict

{'movie_title': 'Top Gun (1986)',
 'movie_summary': 'The Top Gun Naval Fighter Weapons School is where the best of the best train to refine their elite flying skills. When hotshot fighter pilot Maverick is sent to the school, his reckless attitude and cocky demeanor put him at odds with the other pilots, especially the cool and collected Iceman.',
 'domestic_gross': 180258178,
 'international_gross': 177030000,
 'worldwide_gross': 357288178,
 'Domestic Distributor': 'Paramount Pictures',
 'Domestic Opening': 8193052,
 'Earliest Release Date': '1986-05-16',
 'Running Time': 109,
 'Genres': 'Action Drama'}

In [83]:
a = []

In [ ]:
import pandas as pd

df = pd.DataFrame(lista_geral)
df.pop('Budget')
df.pop('MPAA')

df.rename(columns={'Domestic Distributor': 'domestic_distributor'}, inplace=True)
df.rename(columns={'Domestic Opening': 'domestic_opening'}, inplace=True)
df.rename(columns={'Earliest Release Date': 'release_date'}, inplace=True)
df.rename(columns={'Running Time': 'running_time'}, inplace=True)
df.rename(columns={'Genres': 'genres'}, inplace=True)

df = df[['movie_title', 'movie_summary', 'release_date', 'genres', 'running_time', 'domestic_gross', 'international_gross', 'total_gross', 'domestic_opening', 'domestic_distributor']]

df.to_csv(f'worldwide_box_office_{year}.csv', index=False)

In [194]:
import pandas as pd

df = pd.DataFrame(lista_geral)
df.pop('Budget')
df.pop('MPAA')

0        NaN
1      PG-13
2        NaN
3          R
4        NaN
       ...  
194      NaN
195      NaN
196      NaN
197      NaN
198      NaN
Name: MPAA, Length: 199, dtype: object

In [195]:
df.rename(columns={'Domestic Distributor': 'domestic_distributor'}, inplace=True)
df.rename(columns={'Domestic Opening': 'domestic_opening'}, inplace=True)
df.rename(columns={'Earliest Release Date': 'release_date'}, inplace=True)
df.rename(columns={'Running Time': 'running_time'}, inplace=True)
df.rename(columns={'Genres': 'genres'}, inplace=True)

In [196]:
df = df[['movie_title', 'movie_summary', 'release_date', 'genres', 'running_time', 'domestic_gross', 'international_gross', 'total_gross', 'domestic_opening', 'domestic_distributor']]

In [197]:
df

,movie_title,movie_summary,release_date,genres,running_time,domestic_gross,international_gross,total_gross,domestic_opening,domestic_distributor
0,Batman (1989),The Dark Knight of Gotham City begins his war ...,1989-06-23,Action Adventure,126,251409241,160160000.0,411569241,40489746.0,Warner Bros.
1,Indiana Jones and the Last Crusade (1989),"In 1938, after his father goes missing while p...",1989-05-24,Action Adventure,127,197171806,277000000.0,474171806,29355021.0,Paramount Pictures
2,Dead Poets Society (1989),Maverick teacher John Keating returns in 1959 ...,1989-06-02,Comedy Drama,128,95860116,140000000.0,235860116,340456.0,Walt Disney Studios Motion Pictures
3,Lethal Weapon 2 (1989),Riggs and Murtaugh are on the trail of South A...,1989-07-07,Action Crime Thriller,114,147253986,80600000.0,227853986,20388800.0,Warner Bros.
4,Look Who's Talking (1989),"After a single, career-minded woman is left on...",1989-10-13,Comedy,93,140088813,156911000.0,296999813,12107784.0,TriStar Pictures
...,...,...,...,...,...,...,...,...,...,...
194,Powwow Highway (1989),Two Northern Cheyenne men take a road trip fro...,1989-02-24,Adventure Drama,91,283747,NaN,283747,10627.0,Warner Bros.
195,Kill Me Again (1989),A young detective becomes involved with a beau...,1989-10-27,Action Crime Drama Thriller,94,283694,NaN,283694,66013.0,InterStar Releasing
196,Heavy Petting (1989),Celebrity confessions about their first sexual...,1989-09-22,Documentary,74,272371,NaN,272371,69007.0,Skouras Pictures
197,Paperhouse (1988),A young girl lost in the loneliness and boredo...,1989-02-17,Drama Fantasy,92,241278,NaN,241278,6687.0,Vestron Pictures


In [198]:
df.to_csv(f'worldwide_box_office_{year}.csv', index=False)

In [201]:
driver.quit()

In [168]:
df = pd.read_csv(f'worldwide_box_office_1986.csv')